In [19]:
import numpy as np
import scipy.stats as stats

# data columns: Time(s) Height(mm) Diameter(mm) Length(mm) Observer(0|1)
D = np.loadtxt("../data/measurements.txt")


# prepare data for analysis
v = 20000  # 20 ml target volume
g = 9820  # acceleration due to gravity in mm/s
nu = 1.53  # kinematic vicosity, mu/rho, mm^2/s
M = []
for [t, h, d, l] in D[:, 0:4]:
    M.append([(v/t)/((l**2)*((l*g)**0.5)), (h/l), (d/l), (nu/(l*((l*g)**0.5)))])
M = np.array(M)

# Model specification matrix, replace column 1 with ones, take natural log of values
X = np.log(M[:, 0:4])
X[:, 0] = 1
X = np.c_[X, D[:, 4]]  # add observer indicator variable

# Realization of response vector
y = np.log(M[:, 0])

for i, yi in enumerate(y):
    print(f"{yi} = {X[i]}")

-17.04768667660043 = [  1.          -0.58434432  -6.89632658 -14.32356028   0.        ]
-17.043179306214142 = [  1.          -0.58434432  -6.89632658 -14.32356028   0.        ]
-17.043179306214142 = [  1.          -0.58434432  -6.89632658 -14.32356028   0.        ]
-11.182193210214665 = [  1.           1.24250647  -5.06947579 -11.5832841    0.        ]
-11.184429990185434 = [  1.           1.24250647  -5.06947579 -11.5832841    0.        ]
-11.201048240789785 = [  1.           1.24250647  -5.06947579 -11.5832841    0.        ]
-11.128809980461135 = [  1.           1.24250647  -5.06947579 -11.5832841    1.        ]
-11.144379056976163 = [  1.           1.24250647  -5.06947579 -11.5832841    1.        ]
-11.155447147167221 = [  1.           1.24250647  -5.06947579 -11.5832841    1.        ]
-15.523253296817732 = [  1.          -0.58434432  -6.45368247 -14.32356028   0.        ]
-15.554249757189066 = [  1.          -0.58434432  -6.45368247 -14.32356028   0.        ]
-15.41671123542705 = [

### Fysisk modell

$\frac{\langle Q\rangle}{l^2\sqrt{lg}}=\Phi(\frac{h}{l}, \frac{d}{l}, \frac{\mu}{\rho l\sqrt{lg}})$

där $l$ är ett rörs längd, $d$ dess diameter, $h$ är vattenreservoarens höjd över röret, $g$ är accelerationen pga gravitation, $\mu$ är vätskans viskositet och $\rho$ dess densitet. $Q$ är den uppmätta flödeshastigheten. Detta har räknats fram med Buckinghams Teorem, en dimensionsanalysteknik från fysiken.

Genom att ansätta nya variabler istället för kvoterna och anta en potensform kan uttrycket skrivas om på formen

\begin{equation}
q = \alpha k^ar^bm^c.
\end{equation}

Linearisering genom att ta $\ln$ i båda led:

\begin{equation*}
    \ln q = \ln \alpha + a \ln k + b \ln r + c \ln m.
\end{equation*}
Vi kan se detta som formen:
\begin{equation}
    \mu_{Y|x_1,x_2,x_3} = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3
\end{equation}
alltså
\begin{equation*}
    \mathbf{Y} = X\boldsymbol\beta + \mathbf{E},
\end{equation*}
där $X$ är modellspecifikations matrisen. $\mathbf{E}$ är vektorn med stokastiska fel med avseende på väntevärdet och $\mathbf{Y}$ är en respons vektor för $Y$ över hela stickprovet. 

Normalform: 
\begin{equation*}
    (X'X)\mathbf{b} = X'\mathbf{y},
\end{equation*}
där $\mathbf{y}$ är vektorn av observeraded värden i respons vektorn $\mathbf{Y}$ och $\mathbf{b}$ är den sökta vektorn med de uppskattade modell parametrarna. OLS för $\boldsymbol\beta$ blir,
\begin{equation}
    \boldsymbol{\hat{\beta}} = \mathbf{b} = (X'X)^{-1}X'\mathbf{y}.
\end{equation}

### Regression och statistik

In [28]:
# Multiple linear regression, least squares estimate for ^B
b = np.linalg.inv(X.T @ X) @ X.T @ y


array([-2.58376909,  0.87000771,  3.60315069, -0.75188958,  0.0168805 ])

Vi räknar ut ett antal statistiska värden:

$v$ antalet parametrar i modellen (number of features)

$n$ stickprovets storlek (number of samples, sample size)

SSE Sum of Squared Errors aka Residual Sum of Squares

\begin{equation}
\sum_{i=1}^n[y_i - (b_0 + b_1 x_{1i} + ... + b_k x_{ki})]²
\end{equation}

Vilket alltså betyder att vi drar ifrån högerledet från vänsterledet på varje rad, kvadrerar och sedan lägger ihop alla resultaten.

Variansen

$\sigma^2 = \frac{\mathrm{SSE}}{n-v-1}$ 

Standarddeviation

$S = \sqrt{\sigma^2} = \sigma$

SSR Regression Sum of Squares
\begin{equation}
b_0 \sum_{i=1}^n{y_i} + b_1 \sum_{i=1}^n{x_{1i}y_i} + ... + b_v \sum_{i=1}^n{x_{vi} y_i} - \frac{(\sum_{i=1}^n{y_i})²}{n}
\end{equation}

$S_{yy}$ total varians (i responsvektorn Y)

\begin{equation}
\sum_{i=1}^n{y_i²}-\frac{(\sum_{i=1}^n{y_i})²}{n}
\end{equation}



In [21]:
# Sum of Squared Errors, deviation and variance
v = len(b)-1
n = len(M)
SSE = np.sum(np.square(y - (X @ b)))
var = SSE/(n-v-1)
S = np.sqrt(var)

# Regression sum of squares, total variance
SSR = np.sum(b*(X.T @ y)) - (np.square(np.sum(y))/n)
Syy = np.sum(np.square(y)) - (np.square(np.sum(y))/n)


## Signifikanstest
Är regressionen signifikant? Eller är alla $\beta_i$ = 0?

Ett sorts rimlighetstest -- om vi inte klarar detta test är det ingen idé att försöka utröna något ur resultaten.

Vår hypotes $H_0$ är alltså att alla $\beta_i$ är noll.

Om detta är sant följer nedanstående uttryck en F-distribution med $k$ och $n-k-1$ frihetsgrader. Detta värde läses av i en tabell.

\begin{equation}
\frac{\mathrm{SSR}/k}{S²}
\end{equation}
där $S²$ är en uppskattning av $\sigma²$.

Om vi får ett väldigt litet värde är alltså hypotesen falsk, och minst en parameter förklarar någon del av datan.

In [29]:
# Significant regression test, test rejects for large values
sig_statistic = (SSR/v)/var

# P score for H0 in significance test is very close to 0,
# -- we reject the hypothesis that no variable is relevant
p_sig = stats.f.sf(sig_statistic, v, n-v-1)

p_sig

2.1976043471186423e-244

### Förklaringsgrad (coefficient of multiple determination)
När vi vet att regressionen är signifikant är nästa fråga, _hur_ signifikant? Hur mycket av datan täcks av modellen, dvs till vilken grad förklarar modellen datan?
\begin{equation*}
R² = \frac{\mathrm{SSR}}{S_{yy}}
\end{equation*}

Detta är ett värde mellan 0 och 1.

In [30]:
Rsq = SSR/Syy

Rsq

0.9971526071889163

### Modellraffinering

Men vilka parametrar är relevanta? Och vi har ju en variabel som bara indikerar vem som gjort observationen. Vad gör vi med den?

Vi kan testa variablerna en och en!

Test statistika:
\begin{equation}
\frac{\hat{\beta_i}}{S\sqrt{c_{ii}}}
\end{equation}
där $\mathbb{c}_{ii}$ är elementet på rad och kolumn $(i, i)$ en särskild matris, kallad varians-kovarians matrisen.

Om hypotesen att variabeln är 0 är sann följer test-statistikan en $T_{n-k-1}$ distribution. Om värdet är väldigt stort eller väldigt litet avfärdar vi hypotesen och variablen _är_ relevant.

Det vill säga, om observatörs-indikatorn är relevant så har vi ett observatörs-bias i datan!

### Varians/Kovarians matris:

Vi härleder inte uttrycket, men på matrisform är det

\begin{equation}
\mathbb{c} = (X^TX)^{-1}\sigma²
\end{equation}

Variansen $\sigma²$ förekommer på diagonalen i denna matris, och parvis relationer med övriga parametrar (kovarians) fyller ut resten av matrisen. 

In [31]:
# Variance-Covariance matrix
c = np.linalg.inv(X.T @ X) * var

# Is there is no observer bias?
b4_statistic = b[4] / (S*np.sqrt(c[4, 4]))
# Two-sided test (rejects for large or small)
p_b4 = 2 * min(stats.t.cdf(b4_statistic, n-v-1), stats.t.sf(b4_statistic, n-v-1))
print(p_b4)

2.34224112533387e-44


Se där! Det finns ett observatörsbias i datan. 

Vad gör vi då?

I själva verket har vi nu två modeller. Dessa modeller har samma lutning, men olika skärningspunkter!

För observatör A är modellen:
\begin{equation}
\mu_A = (\beta_0 + \beta_4) + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 
\end{equation}
och för B:
\begin{equation}
\mu_B = \beta_0  + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 
\end{equation}

Notera att regressionen har hittat ett värde för $\beta_4$:

In [25]:
print(b[4])

0.01688050411591574


In [26]:
# Split models according to observer
modA = np.append(b[0] + b[4], b[1:4])
modB = b[0:4]
Xr = X[:, 0:4]

Så vilken modell är bäst?



In [36]:
# Normalized error around regression line
Oa = (y - (Xr @ modA))/var
Ob = (y - (Xr @ modB))/var

# Binning for chi^2 test, avoiding empty intervals and many low numbers
# iteratively found through trial and error
binsA, rangesA = np.histogram(Oa, 48, (-20, 21))
p_A, _ = np.histogram(Oa, 48, (-20, 21), density=True)

# chi^2 goodness-of-fit test
# -- rejects for too large values
chi_statisticA = 0
for i in range(0, 48):
    chi_statisticA += (np.square(binsA[i] - n*p_A[i])) / (n*p_A[i])

# p-value very large, we do not reject goodness-of-fit
p_chiA = stats.chi2.sf(chi_statisticA, 47)

binsB, rangesB = np.histogram(Ob, 48, (-20, 21))
p_B, _ = np.histogram(Ob, 48, (-20, 21), density=True)

chi_statisticB = 0
for i in range(0, 48):
    chi_statisticB += np.divide((np.square(binsB[i] - n*p_B[i])), (n*p_B[i]))

# p-value very large, we do not reject goodness-of-fit
p_chiB = stats.chi2.sf(chi_statisticB, 47)
print(p_chiA, p_chiB)

0.9999999986322857 0.9999999937061224


Modellerna är alltså likvärdiga bortom $3\sigma$ så skillnaden är i praktiken långt bortom de 99.7% av variansen som regressionen förklarar. Men tekniskt sett är modell A något bättre.

Om vi återgår till orginalformen och räknar ut det modellerade flödet för varje mätpunkt och jämför med det uppmätta flödet tillsammans med vår regressionslinje får vi följande graf:

![image](../assets/regression.png)

Vad är problemen med denna metod?

Vet vi något om datan? Utan EDA har vi missat viktiga begränsningar i datan! Denna regression gäller absolut _inte_ för generella rör -- datan innehåller bara några få rör i tydliga grupperingar och denna modell gäller inte alls utanför precis de rör som testats och är inte alls generell. Alla rören har liten diameter och trots att det finns många mätningar är de gjorde på ett litet urval av den möjliga parameterrymden. Det finns alltså inte tillräckligt med "signal" i datan för att göra en allmän modell!

Modellen är inte heller alls prediktiv. Vi har inte ens försökt träna och testa för att undersöka hur modellen reagerar på okända värden! I själva verket är modellen värdelös för rör med en diameter som är betydligt större -- tex cm ist för mm. Det finns inte heller mycket vi kan göra åt det -- datan innehåller helt enkelt inga datapunkter som skulle låta regressionen hitta en mer generell lösning.

För att få vettiga resultat från statistiska modeller gäller det att förstå problemet vi skall lösa!